In [32]:
from sqlalchemy import create_engine, ForeignKey, select
from sqlalchemy.ext.hybrid import hybrid_property
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from werkzeug.security import check_password_hash, generate_password_hash
from datetime import datetime
from typing import List, Optional
import os, sys
import re
import secrets
import string
import pandas as pd 
cwd = os.getcwd()
if cwd.endswith(os.path.sep + "webapp") or cwd.endswith("/webapp"):
    os.chdir(os.path.dirname(cwd))
print("Working directory:", os.getcwd())
print(os.listdir())
from webapp.config import BASEDIR, Config
from webapp.data.cantons import CANTONS

Working directory: c:\Users\Admin\OneDrive\Documents\GitHub\SecCom
['.git', '.gitignore', '.pre-commit-config.yaml', '.pylintrc', 'babel.cfg', 'data', 'doc', 'gunicorn.py.dist', 'LICENSE', 'notebooks', 'pyproject.toml', 'README.md', 'requirements.txt', 'seccom', 'webapp', 'wsgi.py']


In [31]:
DB_URI = Config.DB_URI
engine = create_engine(DB_URI, echo=True)
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)
session = Session(engine)

2025-04-17 12:07:50,177 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-17 12:07:50,178 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2025-04-17 12:07:50,178 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-17 12:07:50,180 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("canton")
2025-04-17 12:07:50,180 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-17 12:07:50,181 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("district")
2025-04-17 12:07:50,181 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-17 12:07:50,182 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("commune")
2025-04-17 12:07:50,182 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-17 12:07:50,183 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("question_per_survey")
2025-04-17 12:07:50,183 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-17 12:07:50,184 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("survey")
2025-04-17 12:07:50,184 INFO sqlalchemy.engine.Engine [raw

NoReferencedTableError: Foreign key associated with column 'question_per_survey.question_global_uid' could not find table 'question_global' with which to generate a foreign key to target column 'uid'

In [ ]:
with Session(engine) as session:
    
    # Cantons
    with session.begin():
        for code, lang in CANTONS.items():
            db_canton = Canton(
                code=code,
                name=lang["en"],
                name_de=lang["de"],
                name_fr=lang["fr"],
                name_it=lang["it"],
                name_ro=lang["ro"],
                name_en=lang["en"],
            )
            print(f">>> CREATING: {db_canton.name}")
            session.add(db_canton)
            session.flush()

    # Districts and communes
    with session.begin():
        communes = pd.read_excel(
            os.path.join(BASEDIR, "data", "EtatCommunes.xlsx"),
            index_col=4,
            header=0,
        )
        communes["Canton"] = communes["Canton"].apply(lambda x: "CH-" + x if isinstance(x, str) else None)
        communes["Numéro du district"] = communes["Numéro du district"].apply(lambda x: "B" + str(x).zfill(4))

        for index, row in communes.iterrows():
            db_canton = session.execute(select(Canton).filter_by(code=row["Canton"])).one_or_none()
            if db_canton:
                db_canton = db_canton[0]
            else:
                raise RuntimeError("bleh")

            # Districts
            db_district = session.execute(select(District).filter_by(name=row["Nom du district"])).one_or_none()
            if db_district:
                db_district = db_district[0]
                print(f">>> ALREADY EXISTING: {db_district.name}")
            else:
                db_district = District(
                    code=row["Numéro du district"],
                    name=row["Nom du district"],
                    name_de=row["Nom du district"],
                    name_fr=row["Nom du district"],
                    name_it=row["Nom du district"],
                    name_ro=row["Nom du district"],
                    name_en=row["Nom du district"],
                    canton=db_canton,
                )
                print(f">>> CREATING: {db_district.name}")
                session.add(db_district)
                session.flush()

            # Communes
            db_commune = Commune(
                code=index,
                name=row["Nom de la commune"],
                name_de=row["Nom de la commune"],
                name_fr=row["Nom de la commune"],
                name_it=row["Nom de la commune"],
                name_ro=row["Nom de la commune"],
                name_en=row["Nom de la commune"],
                district=db_district,
            )
            print(f">>> CREATING: {db_commune.name}")
            session.add(db_commune)
            session.flush()

    # Surveys and questions per survey
    with session.begin():
        for year in [1988, 1994, 1998, 2005, 2009, 2017, 2023]:
            db_survey = Survey(
                name=f"GSB{str(year)[2:]}",
                year=year,
            )
            session.add(db_survey)
            session.flush()

            gsb = pd.read_excel(
                os.path.join(BASEDIR, "data", "CodeBook_Cleaned.xlsx"),
                sheet_name=str(year),
                index_col=1,
                header=0,
            )
            for index, row in gsb.iterrows():
                db_question = QuestionPerSurvey(
                    code=index,
                    label=row["label"],
                    survey=db_survey,
                    text_de=row["text_de"],
                    text_fr=row["text_fr"],
                    text_it=row["text_it"],
                    text_ro=row["text_ro"],
                    text_en=row["text_en"],
                )
                session.add(db_question)
                session.flush()


InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper[Canton(canton)]'. Original exception was: When initializing mapper Mapper[Canton(canton)], expression 'District' failed to locate a name ('District'). If this is a class name, consider adding this relationship() to the <class 'webapp.models.Canton.Canton'> class after both dependent classes have been defined.